# Graphing Code to Load Data and Plot It Live

## Imports

In [1]:
import plotly as py
import plotly.tools as tls
from plotly import graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
import pandas as pd
import datetime

import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html

from ast import literal_eval

## Initialising Dash and Plotly

In [2]:
app = dash.Dash(__name__)

#Plotly offline mode
init_notebook_mode(connected=True)

timer = 0

In [3]:
#Function to check if any of the items contain an array
def array_check(item,timer):
    item = item[0]
    print(item)
    if(isinstance(item, str)):
        print(item)
        item = literal_eval(item)
        return round(item[timer],2)
    elif(isinstance(item, list)):

        return round(item[-1],2)
    else:

        return round(float(item),2)

# Loading Data

In [4]:
code = "G10392.0"
path = "Experiments/100 Centre" + "/100 Robots/" + code + "/"
robots = np.load(path + "Robots.npy")

with open(path + "dust.txt", "r") as f:
     dust = json.load(f)

constants = pd.read_excel(path + "Constants.xlsx", index_col=0)

[4.543067272059452, 4.697712697185398, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.736043370209816, 4.740925411604785, 4.759600980827488, 4.805598480066214, 4.865385400546455, 4.928929226874953, 5.002754816139114, 5.0879959948437445, 5.218449915637502, 5.40985392

In [5]:
#the update layout function
def update_layout():
    #contains the graph, the interval timing and the live updating table
    app.layout = html.Div(children=[html.Div(
    [
    dcc.Graph(id = 'live_graph'),
    dcc.Interval(
        id = 'graph-update',
        interval = 100, #milliseconds
        n_intervals = 0
        ),
    ],style = {'display': 'inline-block'} ),
        html.Div(
            [
                dcc.Graph(id = 'live_table'),
                dcc.Interval(
                id = 'variable-update',
                interval = 100, #milliseconds
                n_intervals = 0
                )    
                 ],
        style = {'display': 'inline-block','vertical-align': 'top' }    
    )],style={'width': '100%', 'display': 'inline-block'})

update_layout()

In [6]:
#callback is the updating process
@app.callback(Output('live_graph','figure'),
              [Input('graph-update', 'n_intervals')])

#the actual update for the graph
def update_graph(n):
    #initialising global variables
    global timer
    

    position_robot = store_robots[:,:,timer]
    position_dust = dust[timer]
    #creating scatter plot of robots
    data = go.Scatter(
        x=list(position_robot[0]),
        y=list(position_robot[1]),
        name = 'Robots',
        mode = 'markers',
    )
    
    #creating the plotly figure with the robot data
    fig = go.Figure(
        { "data": data, "layout": go.Layout(yaxis=dict(range=[-1000, 1000]),xaxis = dict(range=[-1000,1000]))
        }
    )

    fig.add_trace(go.Scatter(
          x= position_dust[0],
          y= position_dust[1],
          name='Dust Devil',
          mode = 'markers',
          marker_color='rgba(255, 182, 193, .9)',
      ))
    
    #updating layout with circles and different formatting'''
    fig.update_layout(title="<b>Physics Based Swarm Experiment </b>",
    title_x=0.5,
    xaxis_title="X Position (m)",
    yaxis_title="Y Position (m)",
    margin=dict(
        t=50, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
         # bottom margin: 10px
        l=10, # left margin: 10px
        r=10, # right margin: 10px
    ),
    height=900,width=1150,
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 50
    ),
                      yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 50
    )
                     )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    #incrementing timer
    timer = timer + 1
    
    #updating the layout
    update_layout()
    return fig

#app callback for the table
@app.callback(Output('live_table','figure'),
              [Input('variable-update', 'n_intervals')])

def update_variables(n):
    fig = go.Figure(data=[go.Table(header=dict(values=['<b>Simulation Parameters</b>','<b>Values</b>'],
                            line_color='black',
                            font=dict(color='black', size=17)),
                            cells=dict(values=[[  '<b>Simulation Timestep (h:min:s)</b>','<b>Number of Robots</b>','<b>Communication Range (m)</b>', '<b>Gravitational Constant</b>','<b>Power</b>','<b>Local Multiplier</b>','<b>Max Force (N)</b>', '<b>Max Speed (m/s)</b>', '<b>Minimum Neighbour Average (m)</b>', '<b>Average Cluster Size</b>', ', <b> Density (Robots per 100^2 meter) </b>','<b>Measurement Events Count </b>', ], [str(datetime.timedelta(seconds=timer)),len(store_robots[0,:,timer]),array_check(constants.loc["Communication Range"].values,timer),array_check(constants.loc["G"].values,timer),array_check(constants.loc["Power"].values,timer),array_check(constants.loc["Multiplier"].values,timer),array_check(constants.loc["Max Force"].values,timer),array_check(constants.loc["Max Speed"].values,timer),array_check(constants.loc["Minimum Distance to Neighbours"].values,timer),array_check(constants.loc["Average Cluster Size"].values,timer),(constants.loc["Density"].values),array_check(constants.loc["Measurement Events Count"].values,timer)]],align='center',line_color='black'))])
    #fig.layout['template']['data']['table'][0]['header']['fill']['color']='#636EFA'
    fig.update_layout(width = 650, height = 700,margin=dict(
    t=260, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
        b=10, # bottom margin: 10px
        l=10, # left margin: 10px
        r=10,
    ))
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    return fig

In [ ]:
#runs by default
if __name__ == '__main__':
    #running the app server for the simulator
    app.run_server(debug=False, host = '127.0.0.1')

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
